Objective: Create a Regression Model to predict the life expectancy and understand which factors can help us better forecast the life expectancy.

# 0) Dependencise

In [347]:
import sys
from pathlib import Path

# data processing libraries
import pandas as pd
import numpy as np
import datetime
import re

# modeling 
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, HalvingRandomSearchCV, RandomizedSearchCV
from sklearn.feature_selection import RFE, RFECV, SelectFromModel

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer


# Algorithms
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import IsolationForest, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, LassoCV


# metrics 
from scipy.optimize import least_squares
from sklearn.experimental import enable_halving_search_cv
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, explained_variance_score, r2_score

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm


# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5] #change size of plot
import plotly.express as px

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# 1) Data Acquistion

A quick look at:
1. Data Structure / Data Info
2. Categorical Variables
3. Numerical Variables

Data Dictionary:
- country: the country in which the indicators are from (i.e. United States of America or Congo)
- year: the calendar year the indicators are from (ranging from 2000 to 2015)
- status: whether a country is considered to be 'Developing' or 'Developed' by WHO standards
- life_expectancy: the life expectancy of people in years for a particular country and year

- adult_mortality (Ratio) - the adult mortality rate per 1000 population 
- infant_deaths (Ratio) - number of infant deaths per 1000 population; similar to above, but for infants
- alcohol (Ratio) - a country's alcohol consumption rate measured as liters of pure alcohol consumption per capita

- percentage_expenditure (Ratio) - expenditure on health as a percentage of Gross Domestic Product (gdp)
- hepatitis_b (Ratio): number of 1 year olds with Hepatitis B immunization over all 1 year olds in population
- measles (Number): number of reported Measles cases
- BMI: average Body Mass Index (BMI) of a country's total population

- under-five_deaths (Ratio) - number of people under the age of five deaths per 1000 population
- polio (Ratio) - number of 1 year olds with Polio immunization over the number of all 1 year olds in population
- total_expenditure (Ratio) - government expenditure on health as a percentage of total government expenditure
- diphtheria (Ratio) - Diphtheria tetanus toxoid and pertussis (DTP3) immunization rate of 1 year olds
- hiv/aids (Ratio) - deaths per 1000 live births caused by HIV/AIDS for people under 5; number of people under 5 who die due to HIV/AIDS per 1000 births

- gdp: Gross Domestic Product per capita ($USD)
- population: population of a country

- thinness_1-19_years (Ratio) - rate of thinness among people aged 10-19 
- thinness_5-9_years (Ratio) - rate of thinness among people aged 5-9
- income_composition_of_resources (Ratio) - Human Development Index in terms of income composition of resources (index ranging from 0 to 1)

- schooling (Number) - average number of years of schooling of a population

In [ ]:
life_expectancy_data = pd.read_csv("Life Expectancy Data.csv")

## 1.1) A quick look at the data

In [ ]:
life_expectancy_data.shape

In [ ]:
life_expectancy_data.head()

In [ ]:
life_expectancy_data.columns

In [ ]:
# Trim the spaces in the column names
trimed_col = []
for col in life_expectancy_data.columns:
    trimed_name = col.strip()
    trimed_name = trimed_name.replace("  "," ")

    if trimed_name not in ['BMI', 'HIV/AIDS','GDP']:
        trimed_col.append(trimed_name.title())
    else:
        trimed_col.append(trimed_name)
    
life_expectancy_data.columns = trimed_col

# Note that according to the data dictionary, the first column of thinness represent
# Rate of thinness among people aged 10-19. Hence we should rename the column from '1-19' to '10-19'.
life_expectancy_data.rename(columns = {'Thinness 1-19 Years':'Thinness 10-19 Years'},inplace = True)
life_expectancy_data.columns

In [ ]:
life_expectancy_data.describe()

In [ ]:
life_expectancy_data.isnull().sum()

In [ ]:
IMAGES_PATH = Path() / "images" 
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

life_expectancy_data.hist(bins=50, figsize=(12, 8))
save_fig("attribute_histogram_plots")  
plt.show()

## 1.2) Pandas Profiling

Pandas Profiling generates a global deatiled report about the dataset, including the number of records, the number of features, overall missingness and duplicates. It also includes descriptive statistics for each individual variables and correlation statistics between the variables. Refer to 'original_dataset.html' to see the report.

In [ ]:
import pandas_profiling

profile = life_expectancy_data.profile_report(title='Pandas Profiling Report')
profile.to_file("original_dataset.html")

## 1.3) Data Correction

Looking at the above data distribution, we can immediately tell that some values are off.

- **Percentage Expenditure** as the current health percentage of GDP should be a percentage, it cannot go over the range of [0,100]. While in this dataset it has a range from [0,19479] and a mean of 738, this data should be wrong.
- **BMI** as a obesity indicateor usually range from [17,30], again this data is completely off.
- The overall distribution of **GDP** and **Population** seems valid, but they have many small values that do not make sense (for example, some countries have population < 50), and they have over 30% of the missing value.

The good news is all these common information can easily be found online. To improve the data quality, I replaced these features with actual dataset retrieved from WHO, OurWorldInData, and DataWorldBank.


In [ ]:
df = life_expectancy_data.copy()

In [ ]:
pop = pd.read_csv("Population-DataWorldBank.csv")

df = pd.merge(df, pop, how='left', on = ['Country', 'Year'])
df = df.drop(columns = 'Population')
df = df.rename(columns={'Value':'Population'})

df.isnull().sum()

The number of null value did not change, it is possible that the country names are different in the dataset.

In [ ]:
df['Country'][df['Population'].isnull()].unique()

Use country_converter to unify the country name.

In [ ]:
import country_converter as coco

df = life_expectancy_data.copy()
df['Country'] = coco.convert(names = df['Country'], to='name_short')

df['Country'].unique()

In [ ]:
import country_converter as coco

pop = pd.read_csv("Population-DataWorldBank.csv")
pop['Country'] = coco.convert(names = pop['Country'], to='name_short')

df = pd.merge(df, pop, how='left', on = ['Country', 'Year'])
df = df.drop(columns = 'Population')
df = df.rename(columns={'Value':'Population'})

In [ ]:
df['Population'].isnull().sum()

The number of null value significantly decrease! The assumption on the difference between country names was correct.

Coundct the same steps for all the datasets.

In [ ]:
# GDP per capital
gdp = pd.read_csv("GDP-DataWorldBank.csv")
gdp['Country'] = coco.convert(names = gdp['Country'], to='name_short')

df = pd.merge(df, gdp, how='left', on = ['Country', 'Year'])
df = df.drop(columns = 'GDP')
df = df.rename(columns={'Value':'GDP'})

In [ ]:
df['GDP'].isnull().sum()

In [ ]:
# Current Health Expenditure (% of GDP)
che = pd.read_csv("CHE-WHO.csv")
che['Country'] = coco.convert(names = che['Country'], to='name_short')

df = pd.merge(df, che, how='left',on = ['Country', 'Year'])

df = df.drop(columns = {'Percentage Expenditure','Indicator','ParentLocation'})
df = df.rename(columns={'Value':'Percentage Expenditure'})

In [ ]:
df['Percentage Expenditure'].isnull().sum()

It seems like the number of null values increased. However, according to the panda profiiling report, there are over 20% of the values are zero, which means over 600 records were pure zero.

In [ ]:
df['Country'][df['Percentage Expenditure'].isnull()].unique()

In [ ]:
# Body Mass Index
bmi = pd.read_csv("BMI-OurWorldInData.csv")
bmi['Country'] = coco.convert(names = bmi['Country'], to='name_short')

df = pd.merge(df, bmi, how="left", on = ['Country', 'Year'])
df = df.drop(columns = {'BMI','Mean BMI (female)','Mean BMI (male)'})
df = df.rename(columns= {'Mean BMI':'BMI'})

In [ ]:
df['BMI'].isnull().sum()

Similarily, I also found the dataset for those features that have a large number of null values or zeros:
- Infant Deaths: 28.9% zeros        ~= 875 records
- Under-Five Deaths: 26.7% Zeros    ~=784 records
- Hepatitis B: 18.8% Missing Values ~=552 records
- Measles: 33.5% zeros              ~=984 records

In [ ]:
# Infant Deaths
infant = pd.read_csv("InfantMortalityPer1000-DataWorldBank.csv")
infant['Country'] = coco.convert(names = infant['Country'], to='name_short')

df = pd.merge(df, infant, how='left', on = ['Country', 'Year'])
df = df.drop(columns = 'Infant Deaths')
df = df.rename(columns={'Value':'Infant Deaths'})

In [ ]:
df['Infant Deaths'].isnull().sum()

In [ ]:
# Deaths Under 5
child = pd.read_csv("InfantMortalityPer1000-DataWorldBank.csv")
child['Country'] = coco.convert(names = child['Country'], to='name_short')

df = pd.merge(df, child, how='left', on = ['Country', 'Year'])
df = df.drop(columns = 'Under-Five Deaths')
df = df.rename(columns={'Value':'Under-Five Deaths'})

In [ ]:
df['Under-Five Deaths'].isnull().sum()

In [ ]:
# Hep B
hepb = pd.read_csv("HepB-IMU-DataWorld.csv")
hepb['Country'] = coco.convert(names = hepb['Country'], to='name_short')

df = pd.merge(df, hepb, how='left', on = ['Country', 'Year'])
df = df.drop(columns = 'Hepatitis B')
df = df.rename(columns={'Value':'Hepatitis B'})

In [ ]:
df['Hepatitis B'].isnull().sum()

It turns out that we do have a lot of missing information regarding Hepatitis B Immutization rate across countries.

In [ ]:
# Measles
measles = pd.read_csv("Measles-OurWorldInData.csv")
measles['Country'] = coco.convert(names = measles['Country'], to='name_short')

df = pd.merge(df, measles, how='left', on = ['Country', 'Year'])
df = df.drop(columns = 'Measles')
df = df.rename(columns={'Value':'Measles'})

In [ ]:
df['Measles'].isnull().sum()

In [ ]:
df.to_csv('new_LifeExp.csv')

## 1.4) A quick look at the new data

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

df.hist(bins=50, figsize=(12, 8))
save_fig("new_attribute_histogram_plots")  
plt.show()

Many features are skewed, we might need to consider standardization/normalization when building the model.

## 1.5) Sample a test set (remain unseen)

### 1.5.1) Method 1:  Randomly split the data using np.randodm.permutation

In [ ]:
def shuffle_and_split_data(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
np.random.seed(123)

train_set, test_set = shuffle_and_split_data(df, 0.2)

print(len(train_set),len(test_set))

In [ ]:
train_set.isnull().sum()

In [ ]:
test_set.isnull().sum()

### 1.5.2) Method 2: Using hash() function

In [ ]:
from zlib import crc32

def is_id_in_test_set(identifier, test_ratio):
    return crc32(np.int64(identifier)) < test_ratio * 2**32

def split_data_with_id_hash(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: is_id_in_test_set(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
df_with_id = df.reset_index()  # adds an `index` column
train_set, test_set = split_data_with_id_hash(df_with_id, 0.2, "index")

In [ ]:
print(len(train_set),len(test_set))

In [ ]:
train_set.isnull().sum()

In [ ]:
test_set.isnull().sum()

### 1.5.3) Method 3: Using sklearn model selection

In [ ]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=123)

In [ ]:
print(len(train_set),len(test_set))

In [ ]:
train_set.isnull().sum()

In [ ]:
test_set.isnull().sum()

# 2) Data Exploration

Again I used panda profiling to explore the following:
- Descriptive Statistics
- Missing Data (%)
- Duplicates (%)
- Correlation


## 2.1) Pandas Profiling

Refer to 'output.html' to see the report.

In [ ]:
import pandas_profiling

profile = train_set.profile_report(title='Pandas Profiling Report')
profile.to_file("output.html")

### 2.2) Correlation HeatMap

In [ ]:
corr_matrix = train_set.corr()

In [ ]:
corr_matrix['Life Expectancy'].sort_values(ascending = False)

In [ ]:
# check if there is any collinearity between variables
plt.figure(figsize=(26,6))
heatmap = sns.heatmap(train_set.corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12)

path = IMAGES_PATH / 'heatmap.png'
plt.savefig(path, dpi=300, bbox_inches='tight')
    

To further explore the relationship between LifeExpectancy and each independent variable, certain data cleaning needs to be done.

# 3) Splitting Training set and Validation Set

First we need to split the dataset into training set and validation set.

In [ ]:
training_set, validation_set = train_test_split(train_set, test_size = 0.2, random_state=123)

In [ ]:
print(len(training_set),len(validation_set))

In [ ]:
training_set.isnull().sum()

In [ ]:
training_set.columns

# 4) Building a baseline Model

For a simple baseline model, select only numeric features with less than 5% missing values.

In [ ]:
baseline_train = training_set[['Life Expectancy', 'Adult Mortality',
       'Alcohol', 'Polio', 'Diphtheria', 'HIV/AIDS',
       'Thinness 10-19 Years', 'Thinness 5-9 Years',
       'Income Composition Of Resources', 'Schooling', 'Population', 'GDP',
       'Percentage Expenditure', 'BMI', 'Infant Deaths', 'Under-Five Deaths']].dropna()

baseline_valid = validation_set[['Life Expectancy', 'Adult Mortality',
       'Alcohol', 'Polio', 'Diphtheria', 'HIV/AIDS',
       'Thinness 10-19 Years', 'Thinness 5-9 Years',
       'Income Composition Of Resources', 'Schooling', 'Population', 'GDP',
       'Percentage Expenditure', 'BMI', 'Infant Deaths', 'Under-Five Deaths']].dropna()


In [ ]:
X_train = baseline_train.drop(columns='Life Expectancy')
y_train = baseline_train['Life Expectancy']

X_valid = baseline_valid.drop(columns='Life Expectancy')
y_valid = baseline_valid['Life Expectancy']

In [ ]:
dummy = DummyRegressor(strategy="mean")
dummy.fit(X_train,y_train)
y_pred = dummy.predict(X_valid)

mean_squared_error(y_pred, y_valid, squared=False)

In [ ]:
-cross_val_score(dummy, X_train, y_train,
                scoring="neg_root_mean_squared_error", cv=10)

# 5) Data Cleaning

1. Removing correlated variables 
2. Dealing with Missing data
3. Outlier Detection
4. Feature engineering (Categorical)
5. Feature Scaling
6. Feature Selection


7. Clustering
9. Imbalanced data

## 5.1) Removing Correlated Variables

According to the heatmap, the following pair of variables are highly correlated:
- "Thinness 10-19 Years" and "Thinness 5-9 Years"
- "Infant Deaths" and "Under-Five Deaths"
- 'Income Composition of Resources" and "Schooling"

For each pair, drop the one that is less correlated with 'Life Expectancy'.

In [ ]:
training_set = training_set.drop(columns={'Thinness 10-19 Years','Under-Five Deaths','Schooling'})
training_set.shape

In [ ]:
training_set.describe()

## 5.2) Dealing with Missing Values

We consider three options for handling the missing values.
1. Drop the records that contain missing values
2. Drop the columns that contain missing values
3. Impute the missing values with mean/median/mode

In [ ]:
# Identify percentage of null values in each column. (Can also be found in output.html)
training_set.isnull().sum()*100/training_set.isnull().count()

In [ ]:
null_rows_idx = training_set.isnull().any(axis=1)
training_set.loc[null_rows_idx].head()

In [ ]:
training_set.loc[null_rows_idx].shape

In [ ]:
# Drop rows with missing value
le_option1 = training_set.copy()
le_option1.dropna(inplace=True)
le_option1.loc[null_rows_idx].head()

In [ ]:
# Drop columns withh missing value
le_option2 = training_set.copy()
le_option2.dropna(axis='columns',inplace=True)  # option 2
le_option2.loc[null_rows_idx].head()

In [ ]:
# Impute the missing values with statistical measures
le_option3 = training_set.copy()
le_option3.fillna(le_option3.mean(), inplace=True)  # or median() / mode()
le_option3.loc[null_rows_idx].head()

Hepatitis B has almost 20% missing value, simply dropping the records will result in huge data loss, therefore we abandon option1.

To see if we can apply option2, check the relationship between Hepatitis B and Life Expectancy.

In [ ]:
corr_matrix['Life Expectancy']['Hepatitis B']

In [ ]:
# Mean Hepatitis B Immunization Rate over Years
training_set.groupby('Year')['Hepatitis B'].mean().plot()

In [ ]:
training_set['Hepatitis B'][training_set['Year']==2003].isnull().count()

It seems like there is a lot of value missing in the year of 2003. But in general it does show a linear relationship between Hepatitis B and Life Expectancy.

Move on to option 3.

In [ ]:
training_set.groupby('Year')['Hepatitis B'].median().plot()

Apply the same strategy to all columns with missing values.

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [ ]:
# Separating out the numerical attributes to use the "median" strategy
training_set_num = training_set.select_dtypes(include=[np.number])

imputer.fit(training_set_num)

In [ ]:
imputer.statistics_

In [ ]:
# Verify if imputer is working well
training_set_num.median().values

In [ ]:
X = imputer.transform(training_set_num)

Now add the imputed value back to the dataframe

In [ ]:
training_set[imputer.feature_names_in_] = X

In [ ]:
training_set.loc[null_rows_idx].head()

training_set.isnull().sum()

## 5.3) Outlier Detection

In [ ]:
isolation_forest = IsolationForest(random_state=42)
outlier_pred = isolation_forest.fit_predict(X)

anomaly_values = training_set.iloc[outlier_pred == -1]

In [ ]:
anomaly_values.shape

Run the following cell to drop outlier

In [ ]:
#le = le.iloc[outlier_pred == 1]
#le.shape

## 5.4) Feature Engineering

### Handling Categorical Attributes

In [ ]:
training_set['Country'].value_counts().plot(kind='bar', fontsize=2)

Country has a lot of unique value, using one-hot encodinig would result in a high dimensionality and could lead to overfitting. 

We could consider other encoding techniques such as Label Encoding. However, considering that we already have features like 'GDP per capital' and 'Population', it is not really necessary to include 'Country' becuase GDP and population likely capture much of the information that 'Country' can provide.

Therefore, simply drop this column.

In [ ]:
training_set = training_set.drop(columns='Country')
training_set.head()

The dataset only contains two different type of country status, therefore we can apply label encoing by assigning a value of 0 to one category and 1 to the other cateogry.

In [ ]:
training_set['Status'].value_counts()

In [ ]:
training_set['Status'] = LabelEncoder().fit_transform(training_set['Status'])
training_set['Status'].value_counts()

## 5.5) Feature Scaling

In [ ]:
X = training_set.loc[:,training_set.columns != 'Life Expectancy']
y = training_set['Life Expectancy']

In [ ]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
le_num_min_max_scaled = min_max_scaler.fit_transform(training_set_num)

In [ ]:
std_scaler = StandardScaler()
le_num_std_scaled = std_scaler.fit_transform(training_set_num)

In [ ]:
X_std = std_scaler.fit_transform(X)

## 5.6) Feature Selection

First use RFECV to determine the optimal number of features

In [ ]:
reg =  LinearRegression()
selector = RFECV(estimator=reg,cv=5)
selector.fit(X_std,y)
X_new = selector.transform(X_std)

X_new

In [ ]:
selector.ranking_

In [ ]:
selector.n_features_

In [ ]:
selected_features = X.columns[selector.support_]
selected_features

In [ ]:
X[selected_features]

Use LASSO to verify

In [ ]:
lasso = LassoCV().fit(X_std, y)

importance_features = lasso.coef_ !=0
importance_features

In [ ]:
X.columns[importance_features]

# 6) Transformation Pipelines

Organize all data preprocessing in step 3 into one pipeline, including:
1. Removing correlated variables
2. Imputing missing values
3. Handling categorical variables
4. Standardization

In [ ]:
col_to_drop = ['Country','Thinness 10-19 Years','Under-Five Deaths','Schooling']

train_set = train_set.drop(columns=col_to_drop)
test_set = test_set.drop(columns=col_to_drop)

In [ ]:
# Imputation and Standardization

num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler())
])

cat_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("label_encoder", FunctionTransformer(LabelEncoder().fit_transform)),
    ("reshape", FunctionTransformer(lambda x: x.reshape(-1,1), validate=False))
])

In [ ]:
# Update the preprocessing pipeline
preprocessing = make_column_transformer(
    (num_pipeline, make_column_selector(dtype_include=np.number)),
    (cat_pipeline, make_column_selector(dtype_include=object))
)

In [ ]:
test = train_set.drop(columns={'Life Expectancy'})

le_prepared = preprocessing.fit_transform(test)
le_prepared[:2]

# 6) Modelling and Evaluating on Traing Set


In [ ]:
training_set, validation_set = train_test_split(train_set, test_size = 0.2, random_state=123)

In [ ]:
X_train = training_set.loc[:,training_set.columns != 'Life Expectancy']
y_train = imputer.fit_transform(training_set[['Life Expectancy']])

X_valid = validation_set.loc[:,validation_set.columns != 'Life Expectancy']
y_valid = imputer.fit_transform(validation_set[['Life Expectancy']])

## 6.1） Linear Regression

In [ ]:
lr = make_pipeline(preprocessing, LinearRegression())
lr.fit(X_train,y_train)

In [ ]:
y_valid_pred = lr.predict(X_valid) 

print("Mean absolute error =", round(mean_absolute_error(y_valid, y_valid_pred), 2)) 
print("Mean squared error =", round(mean_squared_error(y_valid, y_valid_pred), 2)) 
print("Median absolute error =", round(median_absolute_error(y_valid, y_valid_pred), 2)) 
print("Explain variance score =", round(explained_variance_score(y_valid, y_valid_pred), 2)) 
print("R2 score =", round(r2_score(y_valid, y_valid_pred), 2))

In [ ]:
cross_val_score(lr, X=X_train, y=y_train, cv=5)

In [ ]:
lr_rmses = -cross_val_score(lr, X_train, y_train,
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(lr_rmses).describe()

## 6.2) Decision Trees

In [ ]:
tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=333))
tree_reg.fit(X_train, y_train)

In [ ]:
y_valid_pred = tree_reg.predict(X_valid)

print("Mean absolute error =", round(mean_absolute_error(y_valid, y_valid_pred), 2)) 
print("Mean squared error =", round(mean_squared_error(y_valid, y_valid_pred), 2)) 
print("Median absolute error =", round(median_absolute_error(y_valid, y_valid_pred), 2)) 
print("Explain variance score =", round(explained_variance_score(y_valid, y_valid_pred), 2)) 
print("R2 score =", round(r2_score(y_valid, y_valid_pred), 2))

In [ ]:
cross_val_score(tree_reg, X=X_train, y=y_train, cv=5)

In [ ]:
tree_base_rmses = -cross_val_score(tree_reg, X_train, y_train,
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(tree_base_rmses).describe()

Decision Tree Regressor performs better than linear regression model

## 6.3) Random Forest

In [ ]:
forest_reg = make_pipeline(preprocessing,
                           RandomForestRegressor(random_state=33))
forest_reg.fit(X_train, y_train)

In [ ]:
y_valid_pred = forest_reg.predict(X_valid)

print("Mean absolute error =", round(mean_absolute_error(y_valid, y_valid_pred), 2)) 
print("Mean squared error =", round(mean_squared_error(y_valid, y_valid_pred), 2)) 
print("Median absolute error =", round(median_absolute_error(y_valid, y_valid_pred), 2)) 
print("Explain variance score =", round(explained_variance_score(y_valid, y_valid_pred), 2)) 
print("R2 score =", round(r2_score(y_valid, y_valid_pred), 2))

In [ ]:
cross_val_score(forest_reg, X=X_train, y=y_train, cv=5)

In [ ]:
forest_rmses = -cross_val_score(forest_reg, X_train, y_train,
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(forest_rmses).describe()

Even better!!

Try reduce the dimensions and use the features selected before.

In [ ]:
X_train_selected = X_train[selected_features]
X_valid_selected = X_valid[selected_features]

X_train_selected.isnull().sum()

In [ ]:
forest_selected = make_pipeline(preprocessing, RandomForestRegressor())
forest_selected.fit(X_train_selected,y_train)

In [ ]:
y_valid_pred = forest_selected.predict(X_valid_selected) 

print("Mean absolute error =", round(mean_absolute_error(y_valid, y_valid_pred), 2)) 
print("Mean squared error =", round(mean_squared_error(y_valid, y_valid_pred), 2)) 
print("Median absolute error =", round(median_absolute_error(y_valid, y_valid_pred), 2)) 
print("Explain variance score =", round(explained_variance_score(y_valid, y_valid_pred), 2)) 
print("R2 score =", round(r2_score(y_valid, y_valid_pred), 2))

In [ ]:
cross_val_score(forest_selected, X=X_train_selected, y=y_train, cv=5)

In [ ]:
lr_selected_rmses = -cross_val_score(forest_selected, X_train_selected, y_train,
                              scoring="neg_root_mean_squared_error", cv=10)
pd.Series(lr_selected_rmses).describe()

With less feature, the performance of the model remains almost the same. Base on the principle of Occam's razor, the simpler the better. Therefore we shall proceed with less features.

In [ ]:
X_train = X_train_selected
X_valid = X_valid_selected

# 7) Fine Tune the model

## 7.1) Grid Search

In [345]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestRegressor(random_state=33)),
])
param_grid = [
    {'random_forest__max_features': [4, 6, 8]},
    {'random_forest__max_features': [6, 8, 10]},
]
grid_search = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring='neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardize',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001E5F0037850>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(strategy...
                                                                                         ('reshape',
                                                                                          FunctionTransformer(func=<function <lambda> at 0x000001E6171170A0>))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001E616F641F0>)])),
                                       ('random_forest',
                                        RandomForestRegressor(random_state=33))]),
             param_grid=[{'random_forest__max_features': [4, 6, 8]},
                         {'random_forest__max_features': [6, 8, 10]}],
             scoring='neg_root_mean_squared_error')

In [ ]:
display(grid_search.best_params_)

In [ ]:
display(grid_search.best_estimator_)

In [ ]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)

# extra code – these few lines of code just make the DataFrame look nicer
cv_res = cv_res[["param_random_forest__max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "mean_test_score"]]
score_cols = ["split0", "split1", "split2", "mean_test_rmse"]
cv_res.columns = [ "max_features"] + score_cols
cv_res[score_cols] = -cv_res[score_cols].astype(np.float64)

cv_res.head()

## 7.2) Randomized Search

In [344]:
param_distribs = {'random_forest__max_features': randint(low=2, high=20)}

rnd_search = RandomizedSearchCV(
    full_pipeline, param_distributions=param_distribs, n_iter=10, cv=3,
    scoring='neg_root_mean_squared_error', random_state=42)

rnd_search.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('impute',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('standardize',
                                                                                                StandardScaler())]),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x000001E5F0037850>),
                                                                              ('pipeline-2',
                                                                               Pipeline(steps=[('impute',
                                                                                                SimpleImputer(st...
                                                                                                FunctionTransformer(func=<function <lambda> at 0x000001E6171170A0>))]),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x000001E616F641F0>)])),
                                             ('random_forest',
                                              RandomForestRegressor(random_state=33))]),
                   param_distributions={'random_forest__max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001E5D7474550>},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [ ]:
cv_res = pd.DataFrame(rnd_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res = cv_res[["param_random_forest__max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "mean_test_score"]]
cv_res.columns = ["max_features"] + score_cols
cv_res[score_cols] = -cv_res[score_cols].astype(np.float64)
cv_res.head()

Analyze the best models and their errors

In [ ]:
final_model = rnd_search.best_estimator_  # includes preprocessing
feature_importances = final_model["random_forest"].feature_importances_
feature_importances

# 8) Evaluate the model on the Unseen Test Set

In [ ]:
X_test = train_set.loc[:,train_set.columns != 'Life Expectancy']
y_test = imputer.fit_transform(train_set[['Life Expectancy']])

In [ ]:
y_test_pred = final_model.predict(X_test)

final_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
print(final_rmse)


In [ ]:
print("Mean absolute error =", round(mean_absolute_error(y_test, y_test_pred), 2)) 
print("Mean squared error =", round(mean_squared_error(y_test, y_test_pred), 2)) 
print("Median absolute error =", round(median_absolute_error(y_test, y_test_pred), 2)) 
print("Explain variance score =", round(explained_variance_score(y_test, y_test_pred), 2)) 
print("R2 score =", round(r2_score(y_test, y_test_pred), 2))